# Scrape Mars Data: The News

In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

In [2]:
# Set up executable path
executable_path = {'executable_path': 'C:/bin/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### Assign the url and instruct the browser to visit it

In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)
    # NOTE: This line of code accomplishes two things:
    # 1. Searchng for elements with specific combo of tag (div) and attribute (list_text)
        # ex. `ul.item_list` -> HTML as `<ul class="item_list">`
    # 2. Telling browser to wait one second before searching for components

True

In [4]:
# Set up HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')
    # NOTE: CSS works from right to left: returnig last item on list instead of first
        # select_one : first matching element returned will be a <li /> element w/ class=slide + all nested elements within it

In [5]:
# Q: Which HTMLL attribute will we use to scrape the article's title?
# A: class = "content_title" 

In [6]:
slide_elem.find('div', class_='content_title')

<div class="content_title">A New Video Captures the Science of NASA's Perseverance Mars Rover</div>

In [7]:
# Use the parent element to find the first 'a' tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"A New Video Captures the Science of NASA's Perseverance Mars Rover"

### Next add the summary text : Change class to "article_teaser_body"

In [8]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_= 'article_teaser_body').get_text()
news_p

'With a targeted launch date of July 30, the next robotic scientist NASA is sending to the to the Red Planet has big ambitions.'

# Scrape Mars Data: Featured Image

In [9]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [10]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1] # Index : Want broswer to click second button
full_image_elem.click()

In [11]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [12]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')

# BREAK DOWN CODE:
    # 1. An `img` tag is nested within this HTML, so we've included it
    # 2. `.get('src') pulls the link to the image
    
# Code allows us to pull link to image by pointing BeautifulSoup to where image will be, instead of grabbing URL directly.
# When JPL updates image page, our code will still pull the most recent image

In [13]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mar.com/{img_url_rel}' # f-string evaluted at run-time - print statement and variable it holds will be updated frquently as data we are scrapping is live.
img_url


'https://spaceimages-mar.com/image/featured/mars3.jpg'

# Scrape Mars Data: Mars Facts

In [14]:
# Scrape the entire table with Pandas' .read_html() funciton
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns = ['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
# Convert dataframe back into HTML-ready code
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [16]:
# End automated browsing session
browser.quit()